In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from pandas.tseries.offsets import MonthBegin, DateOffset

In [2]:
# Load the merged hourly data (long format: measured_at, group_id, consumption_fwh, eur_per_mwh, group_label)
df = pd.read_csv('data/merged_hourly_for_training_up_to_2023_sep.csv')
df['measured_at'] = pd.to_datetime(df['measured_at'])
# Parse group_label into attributes using regex to handle varying spaces around "|"
df[['macro_region', 'county', 'municipality', 'segment', 'product_type', 'consumption_bucket']] = df['group_label'].str.split(r'\s*\|\s*', expand=True)
df = df.drop('group_label', axis=1)

In [3]:
# Aggregate hourly to monthly sums (consumption) and means (price)
df['month_start'] = df['measured_at'].dt.to_period('M').dt.to_timestamp()
monthly = df.groupby(['group_id', 'month_start', 'macro_region', 'county', 'municipality', 'segment', 'product_type', 'consumption_bucket']).agg(
    monthly_consumption_fwh=('consumption_fwh', 'sum'),
    monthly_avg_price=('eur_per_mwh', 'mean')
).reset_index()
monthly = monthly.sort_values(['group_id', 'month_start'])

/tmp/ipykernel_132780/3068664848.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['month_start'] = df['measured_at'].dt.to_period('M').dt.to_timestamp()


In [4]:
# Add calendar features
def add_month_time_features(m):
    m = m.copy()
    m['year'] = m['month_start'].dt.year
    m['month'] = m['month_start'].dt.month
    m['is_winter'] = m['month'].isin([12, 1, 2]).astype(int)
    month_dummies = pd.get_dummies(m['month'], prefix='month', drop_first=True)
    m = pd.concat([m, month_dummies], axis=1)
    return m

monthly = add_month_time_features(monthly)

# Encode categorical group attributes
le_macro = LabelEncoder()
monthly['macro_region_encoded'] = le_macro.fit_transform(monthly['macro_region'])
le_segment = LabelEncoder()
monthly['segment_encoded'] = le_segment.fit_transform(monthly['segment'])
le_product = LabelEncoder()
monthly['product_type_encoded'] = le_product.fit_transform(monthly['product_type'])
le_bucket = LabelEncoder()
monthly['consumption_bucket_encoded'] = le_bucket.fit_transform(monthly['consumption_bucket'])

# Drop unnecessary string columns (keep 'county' for weather merging, drop others)
monthly = monthly.drop(['municipality', 'segment', 'product_type', 'consumption_bucket'], axis=1)

In [5]:
# Load weather data
df_weather = pd.read_csv('data/merged_weather_for_training_up_to_2023_sep.csv')
df_weather['timestamp'] = pd.to_datetime(df_weather['timestamp'])
df_weather['month_start'] = df_weather['timestamp'].dt.to_period('M').dt.to_timestamp()
df_weather = df_weather.rename(columns={'region': 'county'})
# Aggregate to monthly avg temp per county
monthly_weather = df_weather.groupby(['county', 'month_start'])['temperature'].mean().reset_index().rename(columns={'temperature': 'avg_temp'})
# Merge with monthly consumption data
monthly = pd.merge(monthly, monthly_weather, on=['county', 'month_start'], how='left')
# For forecasting: Compute seasonal averages (avg temp per county per month from historical data, excluding 2024 to avoid leakage)
# Since filtered data already excludes 2024, use df_weather directly
seasonal_avg = df_weather.groupby(['county', df_weather['timestamp'].dt.month])['temperature'].mean().reset_index()
seasonal_avg = seasonal_avg.rename(columns={'timestamp': 'month', 'temperature': 'avg_temp'})
# Drop 'county' now that weather is merged (but keep for forecasting prep)
monthly = monthly.drop('county', axis=1)

/tmp/ipykernel_132780/3943505639.py:4: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_weather['month_start'] = df_weather['timestamp'].dt.to_period('M').dt.to_timestamp()


In [6]:
# Add lag and rolling features
def add_monthly_lags(m, group_col='group_id'):
    m = m.sort_values([group_col, 'month_start']).copy()
    m['cons_lag_1m'] = m.groupby(group_col)['monthly_consumption_fwh'].shift(1)
    m['cons_lag_12m'] = m.groupby(group_col)['monthly_consumption_fwh'].shift(12)
    m['cons_roll_3m'] = m.groupby(group_col)['monthly_consumption_fwh'].rolling(3, min_periods=1).mean().reset_index(level=0, drop=True)
    return m

monthly = add_monthly_lags(monthly)
monthly = monthly.dropna().reset_index(drop=True)  # Drop rows with missing lags

# Create target: next month's consumption
monthly = monthly.sort_values(['group_id', 'month_start'])
monthly['target_next'] = monthly.groupby('group_id')['monthly_consumption_fwh'].shift(-1)
monthly = monthly.dropna(subset=['target_next'])

In [7]:
# Feature columns (excluding group_id and macro_region as they're encoded)
feature_cols = [
    'year', 'is_winter', 'monthly_avg_price', 'cons_lag_1m', 'cons_lag_12m', 'cons_roll_3m',
    'macro_region_encoded', 'segment_encoded', 'product_type_encoded', 'consumption_bucket_encoded',
    'avg_temp'
] + [col for col in monthly.columns if col.startswith('month_') and col[6:].isdigit()]

X = monthly[feature_cols].copy()
y = monthly['target_next'].copy()

# Explicitly select only numeric and boolean dtypes to avoid dtype promotion errors (e.g., exclude any datetime)
X = X.select_dtypes(include=['number', 'bool'])

# Split: last 12 months as validation
cutoff = monthly['month_start'].max() - pd.DateOffset(months=12)
train_mask = monthly['month_start'] <= cutoff
val_mask = monthly['month_start'] > cutoff
X_train, y_train = X[train_mask], y[train_mask]
X_val, y_val = X[val_mask], y[val_mask]

In [8]:
# Prepare datasets
train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val)

# Parameters (optimized for MAPE as per scoring)
params = {
    'objective': 'regression',
    'metric': 'mape',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'verbose': -1
}

# Train with early stopping
model_monthly = lgb.train(
    params,
    train_data,
    num_boost_round=2000,
    valid_sets=[train_data, val_data],
    callbacks=[lgb.early_stopping(stopping_rounds=100)]
)

# Evaluate on validation
pred_val = model_monthly.predict(X_val)
mape = mean_absolute_percentage_error(y_val, pred_val)
print('Validation MAPE:', mape)
rmse = np.sqrt(mean_squared_error(y_val, pred_val))
print('Validation RMSE:', rmse)

# Baseline evaluation (same month last year)
monthly['baseline_next'] = monthly.groupby('group_id')['monthly_consumption_fwh'].shift(12)
val_df = monthly[val_mask].copy()
val_df = val_df.dropna(subset=['baseline_next'])
baseline_mape = mean_absolute_percentage_error(val_df['target_next'], val_df['baseline_next'])
print('Baseline MAPE:', baseline_mape)
fva = 100 * (baseline_mape - mape) / baseline_mape
print('FVA%:', fva)

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[87]	training's mape: 0.0472559	valid_1's mape: 0.124672
Validation MAPE: 0.12467186490568796
Validation RMSE: 248.97686701708815
Baseline MAPE: 0.232296362567034
FVA%: 46.33068571200237


In [9]:
# Explicitly select dtypes for full X
X = X.select_dtypes(include=['number', 'bool'])

# Retrain on all data using best iteration
best_iter = model_monthly.best_iteration or 200
train_all = lgb.Dataset(X, label=y)
final_model_monthly = lgb.train(params, train_all, num_boost_round=best_iter)
print('Final model trained with', best_iter, 'iterations')

Final model trained with 87 iterations


In [10]:
# Prepare historical monthly for forecasting (include necessary attributes)
monthly_for_forecast = df.groupby(['group_id', 'month_start', 'macro_region', 'county']).agg(
    monthly_consumption_fwh=('consumption_fwh', 'sum'),
    monthly_avg_price=('eur_per_mwh', 'mean')
).reset_index()
monthly_for_forecast = add_month_time_features(monthly_for_forecast)
monthly_for_forecast['macro_region_encoded'] = le_macro.transform(monthly_for_forecast['macro_region'])

# Add other encodings (fetch from monthly or df)
group_attrs = monthly[['group_id', 'segment_encoded', 'product_type_encoded', 'consumption_bucket_encoded']].drop_duplicates()
monthly_for_forecast = pd.merge(monthly_for_forecast, group_attrs, on='group_id')

# Merge historical avg_temp
monthly_for_forecast = pd.merge(monthly_for_forecast, monthly_weather, on=['county', 'month_start'], how='left')

# Add lags
monthly_for_forecast = add_monthly_lags(monthly_for_forecast)

In [11]:
# Forecasting setup
last_hist_month = monthly_for_forecast['month_start'].max()  # Should be 2024-09-01
start_forecast_month = last_hist_month + MonthBegin(1)  # 2024-10-01
n_horizon = 12

forecast_rows = []
all_groups = sorted(monthly_for_forecast['group_id'].unique())

for gid in all_groups:
    grp = monthly_for_forecast[monthly_for_forecast['group_id'] == gid].sort_values('month_start')
    if grp.empty:
        continue
    last_row = grp.iloc[-1]
    hist_cons = grp['monthly_consumption_fwh'].tolist()
    hist_months = grp['month_start'].tolist()
    price_future = grp['monthly_avg_price'].mean()  # Proxy for future prices
    last_lag1 = hist_cons[-1]
    last_roll3 = hist_cons[-3:] if len(hist_cons) >= 3 else hist_cons + [0] * (3 - len(hist_cons))
    current_county = last_row['county']
    
    for h in range(n_horizon):
        target_month = start_forecast_month + DateOffset(months=h)
        year = target_month.year
        month = target_month.month
        is_winter = 1 if month in [12, 1, 2] else 0
        cons_lag_1m = last_lag1
        lag12_month = target_month - DateOffset(months=12)
        lag12_idx = next((i for i, m in enumerate(hist_months) if m == lag12_month), None)
        cons_lag_12m = hist_cons[lag12_idx] if lag12_idx is not None else cons_lag_1m
        cons_roll_3m = np.mean(last_roll3)
        # Use seasonal avg for future months
        avg_temp_row = seasonal_avg[(seasonal_avg['county'] == current_county) & (seasonal_avg['month'] == month)]
        avg_temp = avg_temp_row['avg_temp'].values[0] if not avg_temp_row.empty else 0
        month_dict = {f'month_{m}': 1 if m == month else 0 for m in range(2, 13)}
        
        feat = {
            'year': year,
            'is_winter': is_winter,
            'monthly_avg_price': price_future,
            'cons_lag_1m': cons_lag_1m,
            'cons_lag_12m': cons_lag_12m,
            'cons_roll_3m': cons_roll_3m,
            'macro_region_encoded': last_row['macro_region_encoded'],
            'segment_encoded': last_row['segment_encoded'],
            'product_type_encoded': last_row['product_type_encoded'],
            'consumption_bucket_encoded': last_row['consumption_bucket_encoded'],
            'avg_temp': avg_temp
        }
        feat.update(month_dict)
        
        X_f = pd.DataFrame([feat])[feature_cols]
        y_hat = final_model_monthly.predict(X_f)[0]
        
        forecast_rows.append({
            'measured_at': target_month.strftime('%Y-%m-%dT00:00:00.000Z'),
            'group_id': gid,
            'forecast_consumption_fwh': y_hat
        })
        
        # Update for next iteration
        last_lag1 = y_hat
        last_roll3.append(y_hat)
        last_roll3 = last_roll3[-3:]

In [12]:
# Create wide format DF for submission
forecasts_df = pd.DataFrame(forecast_rows)
wide_df = forecasts_df.pivot(index='measured_at', columns='group_id', values='forecast_consumption_fwh').reset_index()
group_order = sorted(all_groups)
wide_df = wide_df[['measured_at'] + group_order]
# Save as CSV with semicolon separator and comma decimal
wide_df.to_csv('data/monthly_12month_forecast_test.csv', sep=';', decimal=',', index=False, encoding='utf-8')
print('Saved data/monthly_12month_forecast_test.csv')
wide_df.head()

Saved data/monthly_12month_forecast_test.csv


group_id,measured_at,28,29,30,36,37,38,39,40,41,...,697,698,705,706,707,708,709,738,740,741
0,2023-10-01T00:00:00.000Z,2803.784089,135.379005,466.859389,352.013574,153.855162,468.898092,153.512797,145.973491,442.069757,...,111.097436,470.803200,155.543418,360.132497,141.439284,402.601094,130.001889,477.175518,111.097436,412.805919
1,2023-11-01T00:00:00.000Z,2985.340661,137.134534,591.031289,385.048828,162.289328,558.531569,162.289328,148.296859,529.976503,...,115.642776,570.300655,162.289328,400.355492,144.346758,421.744506,135.849006,594.526497,115.642776,533.643282
2,2023-12-01T00:00:00.000Z,3402.214946,137.630100,576.438417,360.454187,158.130203,567.944195,154.619523,147.022263,532.833184,...,112.362712,629.911625,156.718606,412.707249,142.953251,427.725093,132.468267,598.198915,111.625328,620.732411
3,2024-01-01T00:00:00.000Z,4465.272966,140.639243,582.886055,362.398145,159.338065,581.675081,159.338065,146.736433,560.956486,...,119.988531,546.401884,159.008467,416.250116,145.436596,433.498092,134.164554,596.376726,119.273580,532.883310
4,2024-02-01T00:00:00.000Z,3432.443279,142.150424,614.707921,368.794686,157.003254,600.607813,157.003254,146.432035,592.578077,...,123.733781,547.971174,156.673656,426.073534,144.394813,439.166388,133.758705,605.909815,123.733781,529.825250
